In [1]:
from network import CNetwork

In [2]:
import pandas as pd
import numpy as np

In [3]:
from utils import verifyDir

In [4]:
name_dataset = "dataset_2"
length_cut = 1000
random_flag = True

In [5]:
auxiliar_path = 'auxiliar_folder/' + name_dataset   + '/'
verifyDir(auxiliar_path)

In [6]:
df_train = pd.read_csv("df_train_" + name_dataset + "_" + str(length_cut) + "_" + str(random_flag) + ".csv")

In [7]:
df_val = pd.read_csv("df_val_" + name_dataset + "_" + str(length_cut) + "_" + str(random_flag) + ".csv")

In [8]:
df_test = pd.read_csv("df_test_" + name_dataset + "_" + str(length_cut) + "_" + str(random_flag) + ".csv")

In [9]:
df_train.head(5)

,Unnamed: 0,text,label
0,0,I saw through the open door way a lonely girl ...,hermanMelville
1,1,It is my father s fatal science No no Giovanni...,nathanielHawthorne
2,2,left on the earth to show that it was all some...,bmBower
3,3,blowing through the cracks of the floor It was...,andrewLang
4,4,alive from fathoms Mr Couthouy Remarks on Cora...,charlesDarwin


In [10]:
from utils.text_processing import get_word_index, get_sequences, get_common_words, pre_process_text

In [11]:
df_train['text'] = df_train['text'].apply(lambda t: pre_process_text(t, remove_stop_words=False, only_stop_words=True))

In [12]:
df_test['text'] = df_test['text'].apply(lambda t: pre_process_text(t, remove_stop_words=False, only_stop_words=True))

In [13]:
df_val['text'] = df_val['text'].apply(lambda t: pre_process_text(t, remove_stop_words=False, only_stop_words=True))

In [19]:
def get_local_features(sequences, word_features):
    measures = ["dgr_n", "btw", "cc", "sp", "sp_std", "accs_h2", "accs_h3"]
    network_features = pd.DataFrame()
    for text in sequences:
        obj = CNetwork(text, model=None, index_word=None, percentages=None, path=auxiliar_path)
        network = obj.create_network()
        local_measure = obj.get_network_measures(network, word_features)
        a_series = pd.Series(local_measure)
        network_features = network_features.append(a_series, ignore_index=True)
        #network_features = np.vstack([network_features, global_measure]) if len(network_features) > 0 else global_measure
    return network_features

In [20]:
texts = df_train["text"]
word_index, index_word = get_word_index(texts)
sequences = get_sequences(texts, word_index)
word_features = get_common_words(sequences)
print(word_features)

{'86': 0, '32': 1, '55': 2, '121': 3, '49': 4, '79': 5, '76': 6, '14': 7, '114': 8, '31': 9, '29': 10, '34': 11, '87': 12}


In [21]:
X_train = get_local_features(sequences, word_features)

Nodes: 123 - Edges: 418
Len features: 143
Nodes: 104 - Edges: 347
Len features: 143
Nodes: 86 - Edges: 281
Len features: 143
Nodes: 102 - Edges: 344
Len features: 143
Nodes: 81 - Edges: 262


# Progress:  123/123
# Progress:  123/123
# Progress:  123/123
# Progress:  104/104
# Progress:  104/104
# Progress:  104/104
# Progress:   86/86
# Progress:   86/86
# Progress:   86/86
# Progress:  102/102
# Progress:  102/102
# Progress:  102/102
# Progress:   81/81
# Progress:   81/81


# Progress:   81/81
# Progress:  111/111
# Progress:  111/111
# Progress:  111/111
# Progress:  103/103
# Progress:  103/103
# Progress:  103/103
# Progress:   94/94
# Progress:   94/94
# Progress:   94/94
# Progress:  107/107
# Progress:  107/107
# Progress:   53/107

Len features: 143
Nodes: 111 - Edges: 351
Len features: 143
Nodes: 103 - Edges: 386
Len features: 143
Nodes: 94 - Edges: 328
Len features: 143
Nodes: 107 - Edges: 407
Len features: 143
Nodes:

# Progress:  107/107
# Progress:   86/86
# Progress:   86/86
# Progress:   86/86
# Progress:  120/120
# Progress:  120/120
# Progress:  120/120
# Progress:  108/108
# Progress:  108/108
# Progress:  108/108
# Progress:  107/107
# Progress:  107/107
# Progress:   10/107

...

# Progress:   68/68
# Progress:  109/109
# Progress:  109/109
# Progress:  109/109
# Progress:  106/106
# Progress:  106/106
# Progress:  106/106
# Progress:   96/96
# Progress:   96/96
# Progress:   96/96
# Progress:  108/108


Len features: 143
Nodes: 108 - Edges: 357
Len features: 143
Nodes: 102 - Edges: 333
Len features: 143
Nodes: 111 - Edges: 365
Len features: 143
Nodes: 80 - Edges: 267
Len features: 143
Nodes: 101 - Edges: 360


# Progress:  108/108
# Progress:  108/108
# Progress:  102/102
# Progress:  102/102
# Progress:  102/102
# Progress:  111/111
# Progress:  111/111
# Progress:  111/111
# Progress:   80/80
# Progress:   80/80
# Progress:   80/80
# Progress:  101/101
# Progress:  101/101
# Progress:    6/101

Len features: 143


# Progress:  101/101


In [22]:
X_train.shape

(60, 143)

In [23]:
X_val = get_local_features(df_val["text"], word_features)

Nodes: 99 - Edges: 371
Len features: 143
Nodes: 88 - Edges: 317
Len features: 143
Nodes: 107 - Edges: 373
Len features: 143
Nodes: 121 - Edges: 388


# Progress:   99/99
# Progress:   99/99
# Progress:   99/99
# Progress:   88/88
# Progress:   88/88
# Progress:   88/88
# Progress:  107/107
# Progress:  107/107
# Progress:  107/107
# Progress:  121/121
# Progress:  121/121
# Progress:  121/121


Len features: 143
Nodes: 109 - Edges: 358
Len features: 143
Nodes: 110 - Edges: 393
Len features: 143
Nodes: 105 - Edges: 395
Len features: 143
Nodes: 100 - Edges: 287


# Progress:  109/109
# Progress:  109/109
# Progress:  109/109
# Progress:  110/110
# Progress:  110/110
# Progress:  110/110
# Progress:  105/105
# Progress:  105/105
# Progress:  105/105
# Progress:  100/100
# Progress:  100/100


# Progress:  100/100
# Progress:  100/100
# Progress:  100/100
# Progress:  100/100
# Progress:  108/108
# Progress:  108/108
# Progress:  108/108
# Progress:   91/91
# Progress:   91/91
# Progress:   91/91
# Progress:  111/111
# Progress:  111/111
# Progress:   11/111

...

# Progress:  111/111
# Progress:   94/94
# Progress:   94/94
# Progress:   94/94
# Progress:  112/112
# Progress:  112/112
# Progress:  112/112
# Progress:   98/98
# Progress:   98/98
# Progress:   98/98
# Progress:   93/93
# Progress:   93/93
# Progress:   15/93

Len features: 143
Nodes: 94 - Edges: 313
Len features: 143
Nodes: 112 - Edges: 402
Len features: 143
Nodes: 98 - Edges: 330
Len features: 143
Nodes: 93 - Edges: 298
Len features: 143


# Progress:   93/93
# Progress:  103/103
# Progress:  103/103
# Progress:  103/103
# Progress:  120/120
# Progress:  120/120
# Progress:  120/120
# Progress:  101/101
# Progress:  101/101
# Progress:  101/101
# Progress:  120/120
# Progress:  120/120
# Progress:    5/120

Nodes: 103 - Edges: 352
Len features: 143
Nodes: 120 - Edges: 397
Len features: 143
Nodes: 101 - Edges: 404
Len features: 143
Nodes: 120 - Edges: 397
Len features: 143


# Progress:  120/120


In [24]:
X_val.shape

(20, 143)

In [25]:
classes = list(df_train['label'])  ## or 'author'
total_classes = list(set(df_train['label']))  ## or author
print("Classes: {}".format(total_classes))
print("Total classes: {}".format(len(total_classes)))
number_books = (df_train[df_train['label'] == total_classes[0]]).shape[0]
print("Total entities for each class in train: {}".format(number_books))
dict_categories = list(set(classes))
dict_categories = {cat: index for index, cat in enumerate(dict_categories)}

Classes: ['whashingtonIrving', 'pgWodehouse', 'allanPoe', 'janeAusten', 'charlesDickens', 'hectorMunro', 'hgWells', 'richardHarding', 'charlesDarwin', 'nathanielHawthorne', 'thomasHardy', 'markTwain', 'bmBower', 'bramStoker', 'andrewLang', 'arthurDoyle', 'zaneGrey', 'horatioAlger', 'henryJames', 'hermanMelville']
Total classes: 20
Total entities for each class in train: 3


In [26]:
y_train = [dict_categories[y] for y in df_train["label"]]

In [27]:
y_val = [dict_categories[y] for y in df_val["label"]]

In [29]:
df_train["label"].unique()

array(['hermanMelville', 'nathanielHawthorne', 'bmBower', 'andrewLang',
       'charlesDarwin', 'thomasHardy', 'markTwain', 'janeAusten',
       'whashingtonIrving', 'pgWodehouse', 'henryJames', 'bramStoker',
       'horatioAlger', 'zaneGrey', 'richardHarding', 'hgWells',
       'allanPoe', 'charlesDickens', 'arthurDoyle', 'hectorMunro'],
      dtype=object)

In [30]:
df_train.isnull().values.any()

False

In [31]:
df_train.head()

,Unnamed: 0,text,label
0,0,"[I, through, the, a, at, a, A, and, with, abou...",hermanMelville
1,1,"[It, is, my, s, No, no, it, was, not, I, I, on...",nathanielHawthorne
2,2,"[on, the, to, that, it, was, all, more, than, ...",bmBower
3,3,"[through, the, of, the, It, was, very, but, he...",andrewLang
4,4,"[from, on, that, he, has, up, on, the, of, fro...",charlesDarwin


## Exploratory data analysis

In [32]:
df_data = X_train.copy()

In [33]:
df_data["label"] = y_train

In [34]:
import seaborn as sns
import matplotlib.pyplot as plt

In [35]:
#plt.close()
#sns.set_style("whitegrid")
#sns.pairplot(df_data, hue="label", height=3)
#plt.show()

In [36]:
#from sklearn.preprocessing import MinMaxScaler
#scaler = MinMaxScaler()
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(with_mean=True, with_std=True)

In [37]:
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

In [38]:
X_train

array([[-0.41118503, -0.60700232, -2.23606798, ...,  1.10109864,
         2.06458358,  1.28450866],
       [-0.97189188,  0.28129376, -0.54136383, ...,  0.50461222,
         0.64817317,  0.26111504],
       [ 1.6447401 , -0.45895297, -1.10626521, ..., -1.03568134,
        -1.29761019, -3.33675336],
       ...,
       [-0.59808731, -0.90310101,  0.30598825, ...,  0.58717661,
         0.39114921,  0.99359109],
       [-1.53259873,  0.4293431 , -2.09484263, ..., -1.56427689,
        -2.04353257, -2.31222268],
       [ 0.89713096, -0.01480493, -0.11768779, ...,  0.04299045,
        -0.85610456, -0.04245396]])

In [39]:
y_train

[19,
 9,
 12,
 14,
 8,
 10,
 10,
 11,
 3,
 0,
 1,
 10,
 14,
 18,
 13,
 12,
 17,
 19,
 11,
 17,
 14,
 16,
 7,
 7,
 7,
 6,
 13,
 2,
 0,
 13,
 9,
 18,
 3,
 4,
 15,
 4,
 5,
 1,
 0,
 1,
 5,
 12,
 17,
 3,
 2,
 11,
 8,
 19,
 16,
 5,
 18,
 8,
 15,
 2,
 9,
 15,
 6,
 16,
 6,
 4]

# Training with Bayes

In [40]:
from classifierv2 import getClassifier, getClassMetrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score

In [41]:
from sklearn.neighbors import KNeighborsClassifier

In [46]:
knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train, y_train)

predicted = knn_clf.predict(X_val)
print(X_train)
print(y_train)
print(y_val)
print(predicted)
#print accuracy_score
print("accuracy : " + str(accuracy_score(y_val, predicted)))

print("micro f-measure " + str(f1_score(y_val, predicted, average='micro')))

[[-0.41118503 -0.60700232 -2.23606798 ...  1.10109864  2.06458358
   1.28450866]
 [-0.97189188  0.28129376 -0.54136383 ...  0.50461222  0.64817317
   0.26111504]
 [ 1.6447401  -0.45895297 -1.10626521 ... -1.03568134 -1.29761019
  -3.33675336]
 ...
 [-0.59808731 -0.90310101  0.30598825 ...  0.58717661  0.39114921
   0.99359109]
 [-1.53259873  0.4293431  -2.09484263 ... -1.56427689 -2.04353257
  -2.31222268]
 [ 0.89713096 -0.01480493 -0.11768779 ...  0.04299045 -0.85610456
  -0.04245396]]
[19, 9, 12, 14, 8, 10, 10, 11, 3, 0, 1, 10, 14, 18, 13, 12, 17, 19, 11, 17, 14, 16, 7, 7, 7, 6, 13, 2, 0, 13, 9, 18, 3, 4, 15, 4, 5, 1, 0, 1, 5, 12, 17, 3, 2, 11, 8, 19, 16, 5, 18, 8, 15, 2, 9, 15, 6, 16, 6, 4]
[8, 0, 17, 4, 12, 2, 3, 7, 5, 18, 11, 15, 19, 14, 10, 6, 9, 16, 13, 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
accuracy : 0.05
micro f-measure 0.05000000000000001


In [47]:
from sklearn.svm import SVC

In [48]:
svc = SVC(kernel='linear', probability=True)
svc.fit(X_train, y_train)

predicted = svc.predict(X_val)
print(X_train)
print(y_train)
print(y_val)
print(predicted)
#print accuracy_score
print("accuracy : " + str(accuracy_score(y_val, predicted)))

print("micro f-measure " + str(f1_score(y_val, predicted, average='micro')))

[[-0.41118503 -0.60700232 -2.23606798 ...  1.10109864  2.06458358
   1.28450866]
 [-0.97189188  0.28129376 -0.54136383 ...  0.50461222  0.64817317
   0.26111504]
 [ 1.6447401  -0.45895297 -1.10626521 ... -1.03568134 -1.29761019
  -3.33675336]
 ...
 [-0.59808731 -0.90310101  0.30598825 ...  0.58717661  0.39114921
   0.99359109]
 [-1.53259873  0.4293431  -2.09484263 ... -1.56427689 -2.04353257
  -2.31222268]
 [ 0.89713096 -0.01480493 -0.11768779 ...  0.04299045 -0.85610456
  -0.04245396]]
[19, 9, 12, 14, 8, 10, 10, 11, 3, 0, 1, 10, 14, 18, 13, 12, 17, 19, 11, 17, 14, 16, 7, 7, 7, 6, 13, 2, 0, 13, 9, 18, 3, 4, 15, 4, 5, 1, 0, 1, 5, 12, 17, 3, 2, 11, 8, 19, 16, 5, 18, 8, 15, 2, 9, 15, 6, 16, 6, 4]
[8, 0, 17, 4, 12, 2, 3, 7, 5, 18, 11, 15, 19, 14, 10, 6, 9, 16, 13, 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
accuracy : 0.05
micro f-measure 0.05000000000000001
